In [3]:
import pandas as pd
import numpy as np

### GroupBy机制

In [4]:
df = pd.DataFrame({'key1' : ['a', 'a', 'b', 'b', 'a'],
                   'key2' : ['one', 'two', 'one', 'two', 'one'],
                   'data1' : np.random.randn(5),
                   'data2' : np.random.randn(5)})
df

,key1,key2,data1,data2
0,a,one,-0.725206,0.030510
1,a,two,-0.093211,-1.627923
2,b,one,1.705293,-0.818618
3,b,two,-1.895793,0.621379
4,a,one,-0.337138,-0.951106


In [5]:
group = df['data1'].groupby(df['key1'])  #含有一些有关分组键df['key1']的中间数据而已
group

In [6]:
group.mean()

key1
a   -0.385185
b   -0.095250
Name: data1, dtype: float64

In [10]:
means = df['data1'].groupby([df['key1'],df['key2']]).mean()
means

key1  key2
a     one    -0.531172
      two    -0.093211
b     one     1.705293
      two    -1.895793
Name: data1, dtype: float64

In [11]:
means.unstack()

key2,one,two
key1,,
a,-0.531172,-0.093211
b,1.705293,-1.895793


In [12]:
states = np.array(['Ohio', 'California', 'California', 'Ohio','Ohio'])
years = np.array([2005,2005,2006,2005,2006])
df['data1'].groupby([states,years]).mean()

California  2005   -0.093211
            2006    1.705293
Ohio        2005   -1.310500
            2006   -0.337138
Name: data1, dtype: float64

In [13]:
df.groupby('key1').mean()

,data1,data2
key1,,
a,-0.385185,-0.849506
b,-0.095250,-0.098620


In [14]:
df.groupby(['key1','key2']).mean()

data1     data2
key1 key2                    
a    one  -0.531172 -0.460298
     two  -0.093211 -1.627923
b    one   1.705293 -0.818618
     two  -1.895793  0.621379

In [15]:
df.groupby(['key1','key2']).size()

key1  key2
a     one     2
      two     1
b     one     1
      two     1
dtype: int64

### 对分组进行迭代

In [16]:
for name,group in df.groupby('key1'):
    print(name)
    print(group)

a
  key1 key2     data1     data2
0    a  one -0.725206  0.030510
1    a  two -0.093211 -1.627923
4    a  one -0.337138 -0.951106
b
  key1 key2     data1     data2
2    b  one  1.705293 -0.818618
3    b  two -1.895793  0.621379


In [17]:
for (k1,k2),group in df.groupby(['key1','key2']):
    print((k1,k2))
    print(group)

('a', 'one')
  key1 key2     data1     data2
0    a  one -0.725206  0.030510
4    a  one -0.337138 -0.951106
('a', 'two')
  key1 key2     data1     data2
1    a  two -0.093211 -1.627923
('b', 'one')
  key1 key2     data1     data2
2    b  one  1.705293 -0.818618
('b', 'two')
  key1 key2     data1     data2
3    b  two -1.895793  0.621379


In [18]:
pieces = dict(list(df.groupby('key1')))
pieces['b']

,key1,key2,data1,data2
2,b,one,1.705293,-0.818618
3,b,two,-1.895793,0.621379


In [19]:
df.dtypes

key1      object
key2      object
data1    float64
data2    float64
dtype: object

In [20]:
grouped = df.groupby(df.dtypes,axis=1)
for dtype,group in grouped:
    print(dtype)
    print(group)

float64
      data1     data2
0 -0.725206  0.030510
1 -0.093211 -1.627923
2  1.705293 -0.818618
3 -1.895793  0.621379
4 -0.337138 -0.951106
object
  key1 key2
0    a  one
1    a  two
2    b  one
3    b  two
4    a  one


### 选取一列或列的子集

In [21]:
df.groupby(['key1','key2'])[['data2']].mean()

data2
key1 key2          
a    one  -0.460298
     two  -1.627923
b    one  -0.818618
     two   0.621379

In [22]:
people = pd.DataFrame(np.random.randn(5, 5), columns=['a', 'b', 'c', 'd', 'e'], index=['Joe', 'Steve', 'Wes', 'Jim','Travis'])
people.iloc[2:3,[1,2]] = np.nan
people

,a,b,c,d,e
Joe,-0.319494,-0.344087,-0.445520,0.032283,-0.990091
Steve,-0.643474,0.671210,2.178897,-1.573286,1.510518
Wes,-1.026008,NaN,NaN,-0.393574,0.993884
Jim,1.108352,1.882863,-0.984496,1.418954,0.448131
Travis,1.193174,-1.833606,-0.057974,0.401413,0.494069


In [23]:
mapping = {'a': 'red', 'b': 'red', 'c': 'blue','d': 'blue', 'e': 'red', 'f' : 'orange'}
by_column = people.groupby(mapping,axis=1)
by_column.sum()

,blue,red
Joe,-0.413236,-1.653671
Steve,0.605611,1.538254
Wes,-0.393574,-0.032124
Jim,0.434459,3.439346
Travis,0.343439,-0.146363


In [24]:
map_series = pd.Series(mapping)
map_series

a       red
b       red
c      blue
d      blue
e       red
f    orange
dtype: object

In [25]:
people.groupby(map_series,axis=1).count()

,blue,red
Joe,2,3
Steve,2,3
Wes,1,2
Jim,2,3
Travis,2,3


In [26]:
people.groupby(len).sum() #一个字符串长度的数组

,a,b,c,d,e
3,-0.237150,1.538777,-1.430015,1.057663,0.451924
5,-0.643474,0.671210,2.178897,-1.573286,1.510518
6,1.193174,-1.833606,-0.057974,0.401413,0.494069


### 根据索引级别分组

In [27]:
columns = pd.MultiIndex.from_arrays([['US', 'US', 'US', 'JP','JP'],[1, 3, 5, 1, 3]],names=['cty', 'tenor'])
hier_df = pd.DataFrame(np.random.randn(4,5),columns=columns)
hier_df

cty          US                            JP          
tenor         1         3         5         1         3
0     -0.315058 -0.711198  1.904917 -0.893243  0.561047
1     -0.173050  0.157008  0.023192 -0.205512  1.926121
2     -1.040121 -2.418597 -0.859947  1.326754 -0.121402
3      0.796001 -2.051043 -0.265600 -0.402414 -2.135796

In [28]:
hier_df.groupby(level='cty',axis = 1).count()

cty,JP,US
0,2,3
1,2,3
2,2,3
3,2,3


##  数据聚合

In [29]:
#quantile可以计算Series或DataFrame列的样本分位数
df

,key1,key2,data1,data2
0,a,one,-0.725206,0.030510
1,a,two,-0.093211,-1.627923
2,b,one,1.705293,-0.818618
3,b,two,-1.895793,0.621379
4,a,one,-0.337138,-0.951106


In [30]:
grouped = df.groupby('key1')
grouped['data1'].quantile(0.9)

key1
a   -0.141997
b    1.345185
Name: data1, dtype: float64

In [31]:
#使用你自己的聚合函数，只需将其传入aggregate或agg方法即可
def peak_to_peak(arr):
    return arr.max() - arr.min()
grouped.agg(peak_to_peak)

,data1,data2
key1,,
a,0.631995,1.658433
b,3.601086,1.439997


In [32]:
grouped.describe()

data1                                                              \
     count      mean       std       min       25%       50%       75%   
key1                                                                     
a      3.0 -0.385185  0.318725 -0.725206 -0.531172 -0.337138 -0.215175   
b      2.0 -0.095250  2.546352 -1.895793 -0.995521 -0.095250  0.805022   

               data2                                                    \
           max count      mean       std       min       25%       50%   
key1                                                                     
a    -0.093211   3.0 -0.849506  0.833872 -1.627923 -1.289514 -0.951106   
b     1.705293   2.0 -0.098620  1.018231 -0.818618 -0.458619 -0.098620   

                          
           75%       max  
key1                      
a    -0.460298  0.030510  
b     0.261379  0.621379

In [ ]:
#自定义聚合函数要比表10-1中那些经过优化的函数慢得多。这是因为在构造中间分组数据块时存在非常大的开销

### 面向列的多函数应用

In [35]:
f1 = open('D:\\Jupyter\\利用Python进行数据分析2\\yjx\\Python-\\chapter10\\data\\tips.csv')
tips = pd.read_csv(f1)
tips['tip_pct'] = tips['tip'] / tips['total_bill']
tips[:6]

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808
5,25.29,4.71,No,Sun,Dinner,4,0.186240


In [34]:
%pwd

'D:\\Jupyter\\利用Python进行数据分析2\\yjx\\Python-\\chapter10'

In [36]:
grouped = tips.groupby(['day','smoker'])
grouped_pct = grouped['tip_pct']
grouped_pct.agg('mean')

day   smoker
Fri   No        0.151650
      Yes       0.174783
Sat   No        0.158048
      Yes       0.147906
Sun   No        0.160113
      Yes       0.187250
Thur  No        0.160298
      Yes       0.163863
Name: tip_pct, dtype: float64

In [38]:
grouped_pct.agg(['mean','std',peak_to_peak])

mean       std  peak_to_peak
day  smoker                                  
Fri  No      0.151650  0.028123      0.067349
     Yes     0.174783  0.051293      0.159925
Sat  No      0.158048  0.039767      0.235193
     Yes     0.147906  0.061375      0.290095
Sun  No      0.160113  0.042347      0.193226
     Yes     0.187250  0.154134      0.644685
Thur No      0.160298  0.038774      0.193350
     Yes     0.163863  0.039389      0.151240

In [39]:
grouped_pct.agg([('foo','mean'),('bar',np.std)])

foo       bar
day  smoker                    
Fri  No      0.151650  0.028123
     Yes     0.174783  0.051293
Sat  No      0.158048  0.039767
     Yes     0.147906  0.061375
Sun  No      0.160113  0.042347
     Yes     0.187250  0.154134
Thur No      0.160298  0.038774
     Yes     0.163863  0.039389

In [40]:
functions = ['count','mean','max']
result = grouped['tip_pct','total_bill'].agg(functions)
result

tip_pct                     total_bill                  
              count      mean       max      count       mean    max
day  smoker                                                         
Fri  No           4  0.151650  0.187735          4  18.420000  22.75
     Yes         15  0.174783  0.263480         15  16.813333  40.17
Sat  No          45  0.158048  0.291990         45  19.661778  48.33
     Yes         42  0.147906  0.325733         42  21.276667  50.81
Sun  No          57  0.160113  0.252672         57  20.506667  48.17
     Yes         19  0.187250  0.710345         19  24.120000  45.35
Thur No          45  0.160298  0.266312         45  17.113111  41.19
     Yes         17  0.163863  0.241255         17  19.190588  43.11

In [41]:
result['tip_pct']

count      mean       max
day  smoker                           
Fri  No          4  0.151650  0.187735
     Yes        15  0.174783  0.263480
Sat  No         45  0.158048  0.291990
     Yes        42  0.147906  0.325733
Sun  No         57  0.160113  0.252672
     Yes        19  0.187250  0.710345
Thur No         45  0.160298  0.266312
     Yes        17  0.163863  0.241255

In [42]:
ftupes = [('Durchschnitt', 'mean'),('Abweichung', np.var)]
grouped['tip_pct','total_bill'].agg(ftupes)

tip_pct              total_bill            
            Durchschnitt Abweichung Durchschnitt  Abweichung
day  smoker                                                 
Fri  No         0.151650   0.000791    18.420000   25.596333
     Yes        0.174783   0.002631    16.813333   82.562438
Sat  No         0.158048   0.001581    19.661778   79.908965
     Yes        0.147906   0.003767    21.276667  101.387535
Sun  No         0.160113   0.001793    20.506667   66.099980
     Yes        0.187250   0.023757    24.120000  109.046044
Thur No         0.160298   0.001503    17.113111   59.625081
     Yes        0.163863   0.001551    19.190588   69.808518

In [43]:
grouped.agg({'tip':np.max,'size':'sum'})

tip  size
day  smoker             
Fri  No       3.50     9
     Yes      4.73    31
Sat  No       9.00   115
     Yes     10.00   104
Sun  No       6.00   167
     Yes      6.50    49
Thur No       6.70   112
     Yes      5.00    40

In [44]:
grouped.agg({'tip_pct':['min','max','mean','std'],'size':'sum'})

tip_pct                               size
                  min       max      mean       std  sum
day  smoker                                             
Fri  No      0.120385  0.187735  0.151650  0.028123    9
     Yes     0.103555  0.263480  0.174783  0.051293   31
Sat  No      0.056797  0.291990  0.158048  0.039767  115
     Yes     0.035638  0.325733  0.147906  0.061375  104
Sun  No      0.059447  0.252672  0.160113  0.042347  167
     Yes     0.065660  0.710345  0.187250  0.154134   49
Thur No      0.072961  0.266312  0.160298  0.038774  112
     Yes     0.090014  0.241255  0.163863  0.039389   40

### 以“没有行索引”的形式返回聚合数据

In [45]:
tips.groupby(['day','smoker'],as_index=False).mean()

,day,smoker,total_bill,tip,size,tip_pct
0,Fri,No,18.420000,2.812500,2.250000,0.151650
1,Fri,Yes,16.813333,2.714000,2.066667,0.174783
2,Sat,No,19.661778,3.102889,2.555556,0.158048
3,Sat,Yes,21.276667,2.875476,2.476190,0.147906
4,Sun,No,20.506667,3.167895,2.929825,0.160113
5,Sun,Yes,24.120000,3.516842,2.578947,0.187250
6,Thur,No,17.113111,2.673778,2.488889,0.160298
7,Thur,Yes,19.190588,3.030000,2.352941,0.163863


## apply：一般性的“拆分－应用－合并”

In [46]:
def top(df,n=5,column='tip_pct'):    #假设你想要根据分组选出最高的5个tip_pct值。首先，编写一个选取指定列具有最大值的行的函数
    return df.sort_values(by=column)[-n:]
top(tips,n=6)

,total_bill,tip,smoker,day,time,size,tip_pct
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
232,11.61,3.39,No,Sat,Dinner,2,0.291990
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345


In [47]:
tips.groupby('smoker').apply(top)

total_bill   tip smoker   day    time  size   tip_pct
smoker                                                           
No     88        24.71  5.85     No  Thur   Lunch     2  0.236746
       185       20.69  5.00     No   Sun  Dinner     5  0.241663
       51        10.29  2.60     No   Sun  Dinner     2  0.252672
       149        7.51  2.00     No  Thur   Lunch     2  0.266312
       232       11.61  3.39     No   Sat  Dinner     2  0.291990
Yes    109       14.31  4.00    Yes   Sat  Dinner     2  0.279525
       183       23.17  6.50    Yes   Sun  Dinner     4  0.280535
       67         3.07  1.00    Yes   Sat  Dinner     1  0.325733
       178        9.60  4.00    Yes   Sun  Dinner     2  0.416667
       172        7.25  5.15    Yes   Sun  Dinner     2  0.710345

In [48]:
tips.groupby(['smoker','day']).apply(top,n=1,column='total_bill')

total_bill    tip smoker   day    time  size   tip_pct
smoker day                                                             
No     Fri  94        22.75   3.25     No   Fri  Dinner     2  0.142857
       Sat  212       48.33   9.00     No   Sat  Dinner     4  0.186220
       Sun  156       48.17   5.00     No   Sun  Dinner     6  0.103799
       Thur 142       41.19   5.00     No  Thur   Lunch     5  0.121389
Yes    Fri  95        40.17   4.73    Yes   Fri  Dinner     4  0.117750
       Sat  170       50.81  10.00    Yes   Sat  Dinner     3  0.196812
       Sun  182       45.35   3.50    Yes   Sun  Dinner     3  0.077178
       Thur 197       43.11   5.00    Yes  Thur   Lunch     4  0.115982

In [50]:
result = tips.groupby('smoker')['tip_pct'].describe()
result

,count,mean,std,min,25%,50%,75%,max
smoker,,,,,,,,
No,151.0,0.159328,0.039910,0.056797,0.136906,0.155625,0.185014,0.291990
Yes,93.0,0.163196,0.085119,0.035638,0.106771,0.153846,0.195059,0.710345


In [51]:
result.unstack('smoker')

       smoker
count  No        151.000000
       Yes        93.000000
mean   No          0.159328
       Yes         0.163196
std    No          0.039910
       Yes         0.085119
min    No          0.056797
       Yes         0.035638
25%    No          0.136906
       Yes         0.106771
50%    No          0.155625
       Yes         0.153846
75%    No          0.185014
       Yes         0.195059
max    No          0.291990
       Yes         0.710345
dtype: float64

In [ ]:
#GroupBy中，当你调用诸如describe之类的方法时，实际上只是应用了下面两条代码的快捷方式而已

### 禁止分组键 分组键会跟原始对象的索引共同构成结果对象中的层次化索引。将group_keys=False传入groupby即可禁止该效果

In [52]:
tips.groupby('smoker',group_keys=False).apply(top)

,total_bill,tip,smoker,day,time,size,tip_pct
88,24.71,5.85,No,Thur,Lunch,2,0.236746
185,20.69,5.00,No,Sun,Dinner,5,0.241663
51,10.29,2.60,No,Sun,Dinner,2,0.252672
149,7.51,2.00,No,Thur,Lunch,2,0.266312
232,11.61,3.39,No,Sat,Dinner,2,0.291990
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345


### 分位数和桶分析

In [53]:
frame = pd.DataFrame({'data1': np.random.randn(1000),'data2': np.random.randn(1000)})
quartiles = pd.cut(frame.data1,4)
quartiles[:10]

0     (-0.371, 1.186]
1     (-0.371, 1.186]
2    (-1.929, -0.371]
3     (-0.371, 1.186]
4    (-1.929, -0.371]
5      (1.186, 2.743]
6     (-0.371, 1.186]
7     (-0.371, 1.186]
8     (-0.371, 1.186]
9    (-1.929, -0.371]
Name: data1, dtype: category
Categories (4, interval[float64]): [(-3.492, -1.929] < (-1.929, -0.371] < (-0.371, 1.186] < (1.186, 2.743]]

In [54]:
def get_stats(group):
    return {'min':group.min(),'max':group.max(),'count':group.count(),'mean':group.mean()}
grouped = frame.data2.groupby(quartiles)
grouped.apply(get_stats).unstack()

,count,max,mean,min
data1,,,,
"(-3.492, -1.929]",25.0,1.657281,0.084725,-1.325116
"(-1.929, -0.371]",327.0,3.275109,-0.087670,-3.074070
"(-0.371, 1.186]",536.0,3.508616,0.042158,-2.899822
"(1.186, 2.743]",112.0,2.090995,-0.053526,-3.846227


In [55]:
grouping = pd.qcut(frame.data1,10,labels=False)
grouped = frame.data2.groupby(grouping)
grouped.apply(get_stats).unstack()

,count,max,mean,min
data1,,,,
0,100.0,1.763766,-0.096341,-3.074070
1,100.0,3.275109,0.005284,-2.552014
2,100.0,2.457643,-0.157079,-2.485250
3,100.0,2.023862,-0.046455,-2.362893
4,100.0,2.970251,0.077651,-2.671367
5,100.0,3.508616,-0.009526,-2.519008
6,100.0,3.158835,0.111674,-2.243058
7,100.0,2.859195,0.113551,-1.564649
8,100.0,2.333932,-0.041883,-2.899822


### 示例：用特定于分组的值填充缺失值

In [56]:
s = pd.Series(np.random.randn(6))
s[::2] = np.nan
s

0         NaN
1   -2.621422
2         NaN
3    0.855157
4         NaN
5   -0.852589
dtype: float64

In [57]:
s.fillna(s.mean())

0   -0.872951
1   -2.621422
2   -0.872951
3    0.855157
4   -0.872951
5   -0.852589
dtype: float64

In [58]:
#假设你需要对不同的分组填充不同的值
states = ['Ohio', 'New York', 'Vermont', 'Florida','Oregon', 'Nevada', 'California', 'Idaho']
group_key = ['East'] * 4 + ['West'] * 4
data = pd.Series(np.random.randn(8),index=states)
data

Ohio          1.119826
New York      0.714415
Vermont       0.020346
Florida      -0.429147
Oregon       -0.797431
Nevada       -1.029615
California    0.004062
Idaho        -0.694606
dtype: float64

In [59]:
data[['Vermont', 'Nevada', 'Idaho']] = np.nan
data

Ohio          1.119826
New York      0.714415
Vermont            NaN
Florida      -0.429147
Oregon       -0.797431
Nevada             NaN
California    0.004062
Idaho              NaN
dtype: float64

In [60]:
data.groupby(group_key).mean()

East    0.468365
West   -0.396685
dtype: float64

In [61]:
fill_mean = lambda g: g.fillna(g.mean())
data.groupby(group_key).apply(fill_mean)

Ohio          1.119826
New York      0.714415
Vermont       0.468365
Florida      -0.429147
Oregon       -0.797431
Nevada       -0.396685
California    0.004062
Idaho        -0.396685
dtype: float64

In [62]:
#也可以在代码中预定义各组的填充值
fill_values = {'East':0.5,'West':-1}
fill_func = lambda g:g.fillna(fill_values[g.name])
data.groupby(group_key).apply(fill_func)

Ohio          1.119826
New York      0.714415
Vermont       0.500000
Florida      -0.429147
Oregon       -0.797431
Nevada       -1.000000
California    0.004062
Idaho        -1.000000
dtype: float64

### 示例：随机采样和排列

In [63]:
suits = ['H', 'S', 'C', 'D']
card_val = (list(range(1, 11)) + [10] * 3) * 4
base_names = ['A'] + list(range(2, 11)) + ['J', 'K', 'Q']
cards = []
for suit in ['H', 'S', 'C', 'D']:
    cards.extend(str(num) + suit for num in base_names)
deck = pd.Series(card_val,index=cards)
deck[:13]

AH      1
2H      2
3H      3
4H      4
5H      5
6H      6
7H      7
8H      8
9H      9
10H    10
JH     10
KH     10
QH     10
dtype: int64

In [64]:
def draw(deck,n=5):             #从整副牌中抽出5张
    return deck.sample(n)
draw(deck)

9C      9
KC     10
3C      3
7H      7
10D    10
dtype: int64

In [65]:
#假设你想要从每种花色中随机抽取两张牌。由于花色是牌名的最后一个字符，所以我们可以据此进行分组
get_suit = lambda card:card[-1]
deck.groupby(get_suit).apply(draw,n=2)

C  3C     3
   KC    10
D  2D     2
   3D     3
H  3H     3
   JH    10
S  9S     9
   JS    10
dtype: int64

In [66]:
deck.groupby(get_suit,group_keys=False).apply(draw,n=2)

4C      4
10C    10
2D      2
10D    10
KH     10
QH     10
10S    10
KS     10
dtype: int64

### 示例：分组加权平均数和相关系数 category计算分组加权平均数

In [67]:
df = pd.DataFrame({'category': ['a', 'a', 'a', 'a', 'b', 'b', 'b', 'b'],'data': np.random.randn(8),'weights': np.random.rand(8)})
df

,category,data,weights
0,a,-0.963581,0.730847
1,a,0.373629,0.609594
2,a,-0.257863,0.135887
3,a,-1.015683,0.291750
4,b,0.099883,0.436627
5,b,-0.309813,0.350328
6,b,2.378074,0.958426
7,b,0.223529,0.373543


In [68]:
grouped = df.groupby('category')
get_wavg = lambda g:np.average(g['data'],weights=g['weights'])
grouped.apply(get_wavg)

category
a   -0.456900
b    1.084409
dtype: float64

In [69]:
f2 = open('D:\Jupyter\利用Python进行数据分析2\yjx\Python-\chapter10\data\stock_px_2.csv')
close_px = pd.read_csv(f2,parse_dates=True,index_col=0)
close_px.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2214 entries, 2003-01-02 to 2011-10-14
Data columns (total 4 columns):
AAPL    2214 non-null float64
MSFT    2214 non-null float64
XOM     2214 non-null float64
SPX     2214 non-null float64
dtypes: float64(4)
memory usage: 86.5 KB


In [70]:
close_px[-4:]

,AAPL,MSFT,XOM,SPX
2011-10-11,400.29,27.00,76.27,1195.54
2011-10-12,402.19,26.96,77.16,1207.25
2011-10-13,408.43,27.18,76.37,1203.66
2011-10-14,422.00,27.27,78.11,1224.58


In [71]:
#计算一个由日收益率（通过百分数变化计算）与SPX之间的年度相关系数组成的DataFrame
spx_corr = lambda x:x.corrwith(x['SPX'])
rets = close_px.pct_change().dropna()
get_year = lambda x:x.year
by_year = rets.groupby(get_year)
by_year.apply(spx_corr)

,AAPL,MSFT,XOM,SPX
2003,0.541124,0.745174,0.661265,1.0
2004,0.374283,0.588531,0.557742,1.0
2005,0.467540,0.562374,0.631010,1.0
2006,0.428267,0.406126,0.518514,1.0
2007,0.508118,0.658770,0.786264,1.0
2008,0.681434,0.804626,0.828303,1.0
2009,0.707103,0.654902,0.797921,1.0
2010,0.710105,0.730118,0.839057,1.0
2011,0.691931,0.800996,0.859975,1.0


In [72]:
by_year.apply(lambda g:g['AAPL'].corr(g['MSFT']))

2003    0.480868
2004    0.259024
2005    0.300093
2006    0.161735
2007    0.417738
2008    0.611901
2009    0.432738
2010    0.571946
2011    0.581987
dtype: float64

### 示例：组级别的线性回归

In [74]:
import statsmodels.api as sm
def regress(data,yvar,xvars):
    Y = data[yvar]
    X = data[xvars]
    X['intercept'] = 1.
    result = sm.OLS(Y,X).fit()
    return result.params
by_year.apply(regress,'AAPL',['SPX'])

,SPX,intercept
2003,1.195406,0.000710
2004,1.363463,0.004201
2005,1.766415,0.003246
2006,1.645496,0.000080
2007,1.198761,0.003438
2008,0.968016,-0.001110
2009,0.879103,0.002954
2010,1.052608,0.001261
2011,0.806605,0.001514


## 透视表pivot table和交叉表crosstab

In [75]:
#小费数据根据day和smoker计算分组平均数（pivot_table的默认聚合类型），并将day和smoker放到行上
tips.pivot_table(index=['day','smoker'])

size       tip   tip_pct  total_bill
day  smoker                                          
Fri  No      2.250000  2.812500  0.151650   18.420000
     Yes     2.066667  2.714000  0.174783   16.813333
Sat  No      2.555556  3.102889  0.158048   19.661778
     Yes     2.476190  2.875476  0.147906   21.276667
Sun  No      2.929825  3.167895  0.160113   20.506667
     Yes     2.578947  3.516842  0.187250   24.120000
Thur No      2.488889  2.673778  0.160298   17.113111
     Yes     2.352941  3.030000  0.163863   19.190588

In [76]:
tips.pivot_table(['tip_pct','size'],index=['time','day'],columns='smoker')

size             tip_pct          
smoker             No       Yes        No       Yes
time   day                                         
Dinner Fri   2.000000  2.222222  0.139622  0.165347
       Sat   2.555556  2.476190  0.158048  0.147906
       Sun   2.929825  2.578947  0.160113  0.187250
       Thur  2.000000       NaN  0.159744       NaN
Lunch  Fri   3.000000  1.833333  0.187735  0.188937
       Thur  2.500000  2.352941  0.160311  0.163863

In [77]:
tips.pivot_table(['tip_pct','size'],index=['time','day'],columns='smoker',margins=True)  #All值为平均数

size                       tip_pct                    
smoker             No       Yes       All        No       Yes       All
time   day                                                             
Dinner Fri   2.000000  2.222222  2.166667  0.139622  0.165347  0.158916
       Sat   2.555556  2.476190  2.517241  0.158048  0.147906  0.153152
       Sun   2.929825  2.578947  2.842105  0.160113  0.187250  0.166897
       Thur  2.000000       NaN  2.000000  0.159744       NaN  0.159744
Lunch  Fri   3.000000  1.833333  2.000000  0.187735  0.188937  0.188765
       Thur  2.500000  2.352941  2.459016  0.160311  0.163863  0.161301
All          2.668874  2.408602  2.569672  0.159328  0.163196  0.160803

In [78]:
tips.pivot_table('tip_pct',index=['time','smoker'],columns='day',aggfunc=len,margins=True) 

day             Fri   Sat   Sun  Thur    All
time   smoker                               
Dinner No       3.0  45.0  57.0   1.0  106.0
       Yes      9.0  42.0  19.0   NaN   70.0
Lunch  No       1.0   NaN   NaN  44.0   45.0
       Yes      6.0   NaN   NaN  17.0   23.0
All            19.0  87.0  76.0  62.0  244.0

In [ ]:
#存在空的组合（也就是NA），你可能会希望设置一个fill_value

### 交叉表：crosstab

In [81]:
from io import StringIO
data = """\
Sample  Nationality  Handedness
1   USA  Right-handed
2   Japan    Left-handed
3   USA  Right-handed
4   Japan    Right-handed
5   Japan    Left-handed
6   Japan    Right-handed
7   USA  Right-handed
8   USA  Left-handed
9   Japan    Right-handed
10  USA  Right-handed"""
data = pd.read_table(StringIO(data), sep='\s+')
data

,Sample,Nationality,Handedness
0,1,USA,Right-handed
1,2,Japan,Left-handed
2,3,USA,Right-handed
3,4,Japan,Right-handed
4,5,Japan,Left-handed
5,6,Japan,Right-handed
6,7,USA,Right-handed
7,8,USA,Left-handed
8,9,Japan,Right-handed
9,10,USA,Right-handed


In [82]:
pd.crosstab(data.Nationality,data.Handedness,margins=True)

Handedness,Left-handed,Right-handed,All
Nationality,,,
Japan,2,3,5
USA,1,4,5
All,3,7,10


In [83]:
pd.crosstab([tips.time,tips.day],tips.smoker,margins=True)

smoker        No  Yes  All
time   day                
Dinner Fri     3    9   12
       Sat    45   42   87
       Sun    57   19   76
       Thur    1    0    1
Lunch  Fri     1    6    7
       Thur   44   17   61
All          151   93  244